In [39]:
import json
import time
import re
import threading

In [63]:
class TweetCache:
    def __init__(self):
        self.cache = {}
        self.refresh_time = 30 * 60
        self.last_refresh_time = time.time()

        self.refresh_thread = threading.Thread(target=self._auto_refresh_cache)
        self.refresh_thread.daemon = True
        self.refresh_thread.start()

    def _refresh_cache(self):
        print("Refreshing cache...")
        self.cache = {}
        self.last_refresh_time = time.time()

    def _normalize_query_input(self, query):
        normalized_values = {key: self._clean_text(str(value).lower()) for key, value in query.items()}
        return normalized_values

    def _clean_text(self, text):
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return cleaned_text.strip()

    def _get_query_id(self, query):
        normalized_query = self._normalize_query_input(query)
        return json.dumps(normalized_query, sort_keys=True)

    def _get_ui_input_concatenated(self, query):
        return ''.join(self._normalize_query_input(query).values())

    def add_to_cache(self, query, data):
        query_id = self._get_query_id(query)
        ui_input_concatenated = self._get_ui_input_concatenated(query)
        self.cache[query_id] = {
            'ID': ui_input_concatenated,
            'Query': query,
            'data': data
        }

    def get_from_cache(self, query):
        query_id = self._get_query_id(query)
        if query_id in self.cache:
            return self.cache[query_id]
        else:
            return None

    def _auto_refresh_cache(self):
        while True:
            current_time = time.time()
            if current_time - self.last_refresh_time >= self.refresh_time:
                self._refresh_cache()
                print("Cache refreshed at:", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(current_time)))
            time.sleep(self.refresh_time / 2)


In [64]:
# Example usage:
cache = TweetCache()

# Add data to cache
query = {'username': 'RED,', 'age': '20'}
data = [{'tweetid': 123, 'username': 'user1'},
        {'tweetid': 456, 'username': 'user2'}]

In [65]:
cache.add_to_cache(query, data)

In [66]:
# Retrieve data from cache
cached_data = cache.get_from_cache(query)
if cached_data:
    print("Data found in cache:")
    print(json.dumps(cached_data, indent=4))
else:
    print("Data not found in cache.")

Data found in cache:
{
    "ID": "red20",
    "Query": {
        "username": "RED,",
        "age": "20"
    },
    "data": [
        {
            "tweetid": 123,
            "username": "user1"
        },
        {
            "tweetid": 456,
            "username": "user2"
        }
    ]
}
